[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/erlingmi/KI_hiof/blob/main/oppgave1/lag_laeringsmaal_og_aktiviteter.ipynb)


# 🇳🇴 Norwegian Curriculum → Learning Goals: Generation + JSON Judge (ChatGPT + Gemini) + Activities + PDF

This Colab notebook:
1) Reads a **Norwegian curriculum** text (`curriculum.txt`),
2) Uses **OpenAI (ChatGPT)** to generate **n** alternative **learning goal** sets,
3) Uses **Gemini** to **score** each set and **rank** them, with the judge returning **pure JSON**,
4) Uses **OpenAI** again to generate **learning activities** for **each goal** in the **top-ranked** set,
5) Exports a **nicely formatted PDF** with the curriculum, top goals, and **full activity text**.


In [14]:
!pip -q install openai google-generativeai python-dotenv tenacity reportlab

In [15]:
import os, json, re, pathlib, csv, time
from datetime import datetime
from getpass import getpass
from typing import List, Tuple, Dict
from tenacity import retry, stop_after_attempt, wait_exponential
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai as genai

load_dotenv(override=False)
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or getpass('Paste your OPENAI_API_KEY (hidden): ')
GEMINI_API_KEY = (os.getenv('GEMINI_API_KEY') or os.getenv('GOOGLE_API_KEY')
                  or getpass('Paste your GEMINI/GOOGLE_API_KEY (hidden): '))
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY
os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY
print('API keys set ✓ (not displayed)')


API keys set ✓ (not displayed)


### (Optional) Mount Google Drive and load keys from `.env`
```
OPENAI_API_KEY=sk-...
GEMINI_API_KEY=AIza...
```

In [16]:
USE_DRIVE = False
if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    env_path = '/content/drive/MyDrive/.env'
    if os.path.exists(env_path):
        load_dotenv(env_path, override=True)
        os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', os.environ.get('OPENAI_API_KEY',''))
        os.environ['GEMINI_API_KEY'] = os.getenv('GEMINI_API_KEY', os.environ.get('GEMINI_API_KEY',''))
        os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', os.environ.get('GEMINI_API_KEY',''))
        print('Loaded keys from Drive .env ✓')
    else:
        print('No .env found in Drive path; using previously provided keys.')


In [17]:
# ---------- CONFIG ----------
N_SUGGESTIONS = 5
GENERATION_MODEL = 'gpt-4o-mini'   # OpenAI (ChatGPT) for generation & activities
JUDGE_MODEL = 'gemini-1.5-pro'     # Google Gemini for judging
TEMPERATURE = 0.7
MAX_TOKENS = 1200
GOALS_MIN, GOALS_MAX = 4, 6
BASE_DIR = pathlib.Path('suggestions'); BASE_DIR.mkdir(parents=True, exist_ok=True)
ALL_SUGGESTIONS_FILE = BASE_DIR / 'all_suggestions.txt'
RANKING_CSV = BASE_DIR / 'ranking.csv'
RANKING_JSON = BASE_DIR / 'ranking.json'
CURRICULUM_PATH = pathlib.Path('curriculum.txt')
print('Configured ✓')


Configured ✓


### Provide your curriculum text file (`curriculum.txt`)

In [18]:
if not CURRICULUM_PATH.exists():
    
    try:
        from google.colab import files
        uploaded = files.upload()
        if uploaded:
            first_name = next(iter(uploaded))
            CURRICULUM_PATH = pathlib.Path(first_name)
            print(f'Using uploaded file: {CURRICULUM_PATH}')
    except Exception:
        print('Running outside Colab or file upload canceled. Ensure curriculum.txt exists.')
assert CURRICULUM_PATH.exists(), f'Curriculum file not found: {CURRICULUM_PATH}'
curriculum_text = CURRICULUM_PATH.read_text(encoding='utf-8')
print(f'Loaded curriculum ({len(curriculum_text)} chars) ✓')


Loaded curriculum (139 chars) ✓


### Static system prompts (hard-coded)

In [19]:
GENERATION_SYSTEM_PROMPT = (
    """
Du er en norsk faglærer som jobber med LK20-kompetansemål. Oppgaven din er å lese en kompetansemåltekst
og bryte den ned til konkrete, observerbare **læringsmål** formulert i klart elevspråk (ca. B1–C1),
passende for vurdering. Målene skal:
  • være spesifikke og målbare (observerbar atferd/produkt)
  • starte med handlingsverb (f.eks. «forklare», «analysere», «drøfte», «utforske», «modellere», «sammenligne»)
  • kunne sjekkes på under 1–2 økter (eller peke mot delmål i lavere vanskelighetsgrad)
  • dekke bredden i kompetansemålet (innhold, ferdigheter, begreper) uten å kopiere setningene
  • sorteres fra **viktigst → minst viktig** for å hjelpe prioritering i undervisningen

*** Eksempel ***
Ta utgangspunkt i følgende kompetansemål fra samfunnsfag etter 10.trinn: utforske hvordan teknologi har vært og fremdeles er en endringsfaktor, og drøfte innvirkningen teknologien har hatt og har på enkeltmennesker, samfunn og natur. Bruk læreplanverket LK20 som ramme for å foreslå ett eller flere læringsmål som:

Er tydelig koblet til formålet med samfunnsfag og relevante kjerneelementer
Er forståelige og meningsfulle for elever på dette trinnet
Er tilpasset elevenes nivå og kontekst
Støtter utvikling av relevante grunnleggende ferdigheter i faget
Kan knyttes til ett eller flere tverrfaglige temaer, dersom det er relevant
Formuler læringsmålene med handlingsorienterte verb (som f.eks. forklare, beskrive, analysere, drøfte, vurdere, sammenligne, bruke), og hold dem konkrete nok til å kunne vurderes. Skriv dem slik at de er forståelige for elever 12 år. 

Eksempel på innspill til KI:

Kompetansemål: "Drøfte hvordan framstillinger av fortiden, hendelser og grupper har påvirket og påvirker folks holdninger og handlinger" (etter 10. trinn)
Støtteinformasjon (automatisk eller manuelt lagt inn):
Formål med faget: Utvikle historisk bevissthet, samfunnsforståelse og evne til å delta i demokratiske prosesser
Kjerneelement: Undring og utforsking, perspektivmangfald og samfunnskritisk tenking
Grunnleggende ferdigheter: Skriving og muntlig formidling av resonnementer, kildekritikk, digitale kilder
Tverrfaglig tema: Demokrati og medborgerskap
Elevgruppe: 10. trinn, heterogen gruppe, tema: holocaust og rasisme i dag
Eksempel på resultat fra KI:

Forslag til læringsmål:

Jeg kan forklare hvordan jødeforfølgelsen under andre verdenskrig fortsatt preger holdninger i samfunnet i dag.
Jeg kan drøfte hvordan film og bøker har påvirket hvordan vi ser på ulike historiske grupper.
Jeg kan bruke kilder for å analysere hvordan framstillinger av samer har endret seg over tid – og hva slags holdninger det kan skape.

*** Eksempel på Output ***
Takk for tydelig instruksjon! Her er et sett med konkrete og elevnære læringsmål som tar utgangspunkt i kompetansemålet:

"Utforske hvordan teknologi har vært og fremdeles er en endringsfaktor, og drøfte innvirkningen teknologien har hatt og har på enkeltmennesker, samfunn og natur."
(samfunnsfag etter 10. trinn, LK20)
🎯 Forslag til læringsmål (tilpasset 12-åringer)
Teknologi og hverdagen min:
Jeg kan beskrive hvordan teknologi påvirker min hverdag hjemme, på skolen og i fritiden.

Jeg kan sammenligne hvordan mine besteforeldre og jeg har brukt teknologi i oppveksten.

Teknologi og samfunnet:
Jeg kan forklare hvordan ny teknologi har forandret jobber og arbeidsplasser i Norge.

Jeg kan drøfte hvordan teknologi kan hjelpe folk – og samtidig skape nye utfordringer i samfunnet.

Teknologi og naturen:
Jeg kan forklare hvordan teknologi har både hjulpet og skadet naturen.

Jeg kan vurdere hvordan vi kan bruke teknologi på en mer bærekraftig måte.

Bruke og forstå informasjon:
Jeg kan bruke artikler, videoer og nettsteder til å finne informasjon om hvordan teknologi har utviklet seg.

Jeg kan vurdere om informasjonen jeg finner på nettet om teknologi er troverdig.

📘 Faglig forankring
Formål med faget samfunnsfag:

Skape forståelse for hvordan samfunnet fungerer, og hvordan mennesker påvirker og blir påvirket av utvikling – spesielt teknologisk utvikling.

Fremme evne til å delta i demokratiske prosesser og forstå bærekraftige valg.

Kjerneelementer fra LK20:

Undring og utforsking: Elevene stiller spørsmål og undersøker teknologiens rolle i samfunnet.

Samfunnskritisk tenking: Elevene ser både fordeler og ulemper ved teknologisk utvikling.

Bærekraftige samfunn: Teknologiens rolle i klima og miljøspørsmål blir sentral.

Grunnleggende ferdigheter:

Muntlig: Presentere og samtale om teknologiens virkninger.

Skriving: Lage rapporter eller tekster om teknologiske endringer.

Digitale ferdigheter: Søke etter informasjon og vurdere kilder.

Lesing: Forstå faglige og samfunnsaktuelle tekster.

Regning: Tolke tall, grafer eller statistikk om teknologi (valgfritt, kontekstavhengig).

Tverrfaglige temaer:

Bærekraftig utvikling: Forholdet mellom teknologi og naturressurser.

Demokrati og medborgerskap: Hvordan teknologi kan styrke eller true rettigheter og deltakelse.

Folkehelse og livsmestring: Hvordan teknologi påvirker helse, sosialt liv og trivsel.

Ønsker du også forslag til vurderingsaktiviteter, tverrfaglige prosjekter eller introduksjonstekst til temaet for elever?

Instruksjoner for UTDATA-format (kun dette – ingen forklaringstekst):
• En nummerert liste med {GOALS_MIN}–{GOALS_MAX} læringsmål, ett per linje.
• Hver linje **kun** selve målet (ingen ekstra metadata).
• Bruk norsk bokmål.
"""
).strip()

JUDGE_SYSTEM_PROMPT = (
    """
Du er sensor og skal KUN vurdere **kvaliteten** på en hel liste med foreslåtte læringsmål
opp mot kompetansemål (relevans, dekningsgrad, presisjon) og didaktiske kriterier (klarhet, målbarhet,
observérbarhet, progresjon, språk for elever, vurderbarhet). Vurder hele settet under ett.

KRAV TIL UTDATA (strengt):
Returner KUN gyldig, minifisert JSON uten kodeblokker, kommentarlinjer eller ekstra tekst.
Eksakt skjema:
{"score": <desimaltall 0–1>, "begrunnelse": "kort setning (maks 1–2 setninger)"}

• `score` må være et tall mellom 0 og 1 (float)
• `begrunnelse` skal være kort og presis
• Ingen annen tekst før eller etter JSON-objektet
""" ).strip()
print('Prompts ready ✓')


Prompts ready ✓


In [20]:
openai_client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
genai.configure(api_key=os.environ['GEMINI_API_KEY'])
gemini_model = genai.GenerativeModel(
    model_name=JUDGE_MODEL,
    system_instruction=JUDGE_SYSTEM_PROMPT,
    generation_config={'response_mime_type': 'application/json'},
)

@retry(stop=stop_after_attempt(4), wait=wait_exponential(multiplier=1, min=1, max=20))
def call_openai_chat(messages, model=GENERATION_MODEL, temperature=0.7, max_tokens=1200):
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

@retry(stop=stop_after_attempt(4), wait=wait_exponential(multiplier=1, min=1, max=20))
def call_gemini_judge(prompt: str):
    return gemini_model.generate_content(prompt)

def generate_one_suggestion(curriculum_text: str) -> str:
    user_prompt = (
        f"Les kompetansemålteksten mellom <KOMPETANSEMAAL>…</KOMPETANSEMAAL> og lag en liste med {GOALS_MIN}–{GOALS_MAX} læringsmål.\n\n"
        f"<KOMPETANSEMAAL>\n{curriculum_text}\n</KOMPETANSEMAAL>"
    )
    messages = [
        {'role':'system','content': GENERATION_SYSTEM_PROMPT},
        {'role':'user',  'content': user_prompt},
    ]
    resp = call_openai_chat(messages, model=GENERATION_MODEL)
    return resp.choices[0].message.content.strip()

def write_text(path: pathlib.Path, text: str):
    path.write_text(text, encoding='utf-8')

def append_text(path: pathlib.Path, text: str):
    with path.open('a', encoding='utf-8') as f:
        f.write(text)

def _extract_json_block(text: str) -> str:
    # Already JSON?
    try:
        json.loads(text); return text
    except Exception: pass
    # Strip code fences
    text = text.strip()
    text = re.sub(r'^```(?:json)?\s*', '', text)
    text = re.sub(r'\s*```$', '', text)
    # First {...}
    s, e = text.find('{'), text.rfind('}')
    if s != -1 and e != -1 and e > s: return text[s:e+1]
    return text

def score_with_gemini_json(suggestion_text: str) -> Dict[str, str]:
    prompt = "VURDER DETTE FORSLAGET (hele listen av læringsmål) og svar KUN i JSON:\n\n" + suggestion_text
    resp = call_gemini_judge(prompt)
    raw = (resp.text or '').strip()
    payload = _extract_json_block(raw)
    try:
        data = json.loads(payload)
        score = float(data.get('score', 0.0))
        note = str(data.get('begrunnelse', ''))
    except Exception:
        score = 0.0
        note = f'Kunne ikke parse JSON. Rått svar: {raw[:500]}'
        data = {'score': score, 'begrunnelse': note}
    return {'score': score, 'begrunnelse': note, 'raw': raw, 'json': data}

def rank_scores(scores: List[float]) -> List[Tuple[int, int, float]]:
    order = sorted(enumerate(scores, start=1), key=lambda x: x[1], reverse=True)
    return [(rank, idx, sc) for rank, (idx, sc) in enumerate(order, start=1)]

print('Helpers ready ✓')


Helpers ready ✓


In [21]:
print('Generating suggestions…')
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
ALL_SUGGESTIONS_FILE.write_text('', encoding='utf-8')
suggestion_paths = []
for i in range(1, N_SUGGESTIONS + 1):
    text = generate_one_suggestion(curriculum_text)
    path = BASE_DIR / f'suggestion_{i:02d}.txt'
    write_text(path, text)
    suggestion_paths.append(path)
    append_text(ALL_SUGGESTIONS_FILE, f'===== SUGGESTION {i} =====\n{text}\n\n')
    print(f'Saved {path}')
print(f'All suggestions consolidated in: {ALL_SUGGESTIONS_FILE}')


Generating suggestions…
Saved suggestions/suggestion_01.txt
Saved suggestions/suggestion_02.txt
Saved suggestions/suggestion_03.txt
Saved suggestions/suggestion_04.txt
Saved suggestions/suggestion_05.txt
All suggestions consolidated in: suggestions/all_suggestions.txt


In [22]:
print('Scoring suggestions with Gemini (JSON)…')
scores, begrunnelser, raws, jsons = [], [], [], []
for path in suggestion_paths:
    s_text = path.read_text(encoding='utf-8')
    result = score_with_gemini_json(s_text)
    scores.append(result['score'])
    begrunnelser.append(result['begrunnelse'])
    raws.append(result['raw'])
    jsons.append(result['json'])
    print(f"{path.name}: score={result['score']:.3f} — {result['begrunnelse']}")

ranking = rank_scores(scores)
print('\nRanking (rank, list_index, score):')
for r in ranking:
    print(r)

with RANKING_CSV.open('w', newline='', encoding='utf-8') as f:
    w = csv.writer(f)
    w.writerow(['rank', 'list_index', 'filename', 'score', 'begrunnelse'])
    for rank, idx, sc in ranking:
        fname = f'suggestion_{idx:02d}.txt'
        w.writerow([rank, idx, fname, f'{sc:.4f}', begrunnelser[idx-1]])

with RANKING_JSON.open('w', encoding='utf-8') as f:
    json.dump({
        'generated': [str(p.name) for p in suggestion_paths],
        'scores': scores,
        'begrunnelser': begrunnelser,
        'ranking': ranking,
        'judge_model': JUDGE_MODEL,
        'generation_model': GENERATION_MODEL,
        'timestamp': timestamp,
        'raw_judge_outputs': raws,
        'json_judge_outputs': jsons,
    }, f, ensure_ascii=False, indent=2)
print(f"\nSaved ranking to:\n- {RANKING_CSV}\n- {RANKING_JSON}")


Scoring suggestions with Gemini (JSON)…
suggestion_01.txt: score=0.850 — Læringsmålene er stort sett relevante, klare og målbare, men dekker ikke alle aspekter ved kompetansemålene. Noe forbedringspotensial mtp. progresjon og vurderbarhet for noen av målene.
suggestion_02.txt: score=0.800 — Læringsmålene er stort sett relevante, klare og målbare, men dekker ikke alle aspekter ved medborgerskap og kunne vært mer spesifikke på observérbarhet og vurderbarhet.
suggestion_03.txt: score=0.850 — Læringsmålene er stort sett relevante, presise og målbare, med god dekningsgrad av typiske kompetansemål knyttet til medborgerskap. Klarhet, progresjon og vurderbarhet er også bra, men kan forbedres ytterligere ved å spesifisere noen av målene mer konkret. Språket er tilpasset elever.
suggestion_04.txt: score=0.850 — Læringsmålene er stort sett relevante, presise og målbare, og dekker sentrale aspekter ved temaet. Det er noe rom for forbedring i observérbarhet og progresjon.
suggestion_05.txt: score=0

## Generate learning activities for the **top-ranked** suggestion set

This cell reads the highest-scoring `suggestion_XX.txt`, parses the learning goals,
and asks **the same OpenAI model** to propose concrete learning activities for **each goal**.

**Output:**
- `suggestions/activities/activities_for_suggestion_XX.md` (all goals aggregated)
- `suggestions/activities/activities_goal_YY.md` (one file per goal)


In [23]:
import re

# Directory for activities
ACTIVITIES_DIR = BASE_DIR / "activities"
ACTIVITIES_DIR.mkdir(parents=True, exist_ok=True)

def generate_activities_for_goal(goal: str, n: int = 5) -> str:
    """Use the same OpenAI model to propose activities for one learning goal.
    Returns Markdown text with a numbered list of activities.
    """
    SYSTEM = (
        "Du er en erfaren norsk faglærer. Du skal foreslå konkrete læringsaktiviteter som hjelper elever å nå et gitt læringsmål. "
        "Lag både individuelle og samarbeidsaktiviteter (bland gjerne). Aktivitetene trenger ikke å være digitale. "
        "For hver aktivitet: gi en kort tittel, skriv en presis beskrivelse, forklar hva elevene lærer, hva læreren bør forberede, og foreslå kort vurdering. "
        "Svar på norsk bokmål og i Markdown som en nummerert liste."
    )
    USER = (
        f"Læringsmål: {goal}\n\n" 
        f"Gi meg {n} forslag til læringsaktiviteter (både individuelle og samarbeidsaktiviteter) som kan hjelpe elevene med å oppnå dette målet. "
        "Forklar hva elevene lærer i aktiviteten, og hva jeg som lærer bør tenke på i forberedelsene."
    )
    messages = [
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": USER},
    ]
    resp = call_openai_chat(messages, model=GENERATION_MODEL, temperature=0.6, max_tokens=1000)
    return resp.choices[0].message.content.strip()

# Determine top-ranked suggestion file
try:
    top_rank, top_idx, top_score = ranking[0]
except Exception:
    # Fallback: load from file if the scoring cell hasn't been run in this session
    data = json.loads(RANKING_JSON.read_text(encoding="utf-8"))
    top_idx = data["ranking"][0][1]
    top_score = data["ranking"][0][2]

top_file = BASE_DIR / f"suggestion_{top_idx:02d}.txt"
assert top_file.exists(), f"Top suggestion file not found: {top_file}"

# Parse goals from the top file (strip numbering like '1) ', '2. ', etc.)
raw_goals = top_file.read_text(encoding="utf-8").splitlines()
goals = []
for line in raw_goals:
    s = line.strip()
    if not s:
        continue
    s = re.sub(r"^\s*\d+\s*[\).:-]?\s*", "", s)
    if s:
        goals.append(s)

agg_path = ACTIVITIES_DIR / f"activities_for_{top_file.stem}.md"
with agg_path.open("w", encoding="utf-8") as f:
    f.write(f"# Læringsaktiviteter for {top_file.name} (score={top_score:.3f})\n\n")
    for i, goal in enumerate(goals, start=1):
        f.write(f"## Læringsmål {i}: {goal}\n\n")
        md = generate_activities_for_goal(goal, n=5)
        f.write(md + "\n\n")
        # Save per-goal file as well
        (ACTIVITIES_DIR / f"activities_goal_{i:02d}.md").write_text(
            f"# Læringsmål: {goal}\n\n" + md + "\n", encoding="utf-8"
        )
        time.sleep(1)

print(f"Saved aggregated activities to: {agg_path}")
print(f"Per-goal files are in: {ACTIVITIES_DIR}")


Saved aggregated activities to: suggestions/activities/activities_for_suggestion_01.md
Per-goal files are in: suggestions/activities


## Export a nicely formatted PDF (full activity text)

This version includes the **full text** of each activity item (title + details),
not just the first line. It parses each numbered activity block and prints all lines.

**Output:** `suggestions/report_learning_goals_activities.pdf`


In [24]:
from reportlab.lib.pagesizes import A4
from reportlab.platypus import (
    SimpleDocTemplate, Paragraph, Spacer, PageBreak,
    ListFlowable, ListItem, KeepTogether
)
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_CENTER
from reportlab.lib.units import cm
from reportlab.lib import colors
from datetime import datetime
from xml.sax.saxutils import escape

import re, json

PDF_PATH = BASE_DIR / 'report_learning_goals_activities.pdf'
ACTIVITIES_DIR = BASE_DIR / 'activities'

# Resolve top-ranked suggestion
try:
    top_rank, top_idx, top_score = ranking[0]
except Exception:
    data = json.loads(RANKING_JSON.read_text(encoding='utf-8'))
    top_idx = data['ranking'][0][1]
    top_score = data['ranking'][0][2]

top_file = BASE_DIR / f'suggestion_{top_idx:02d}.txt'
agg_md = ACTIVITIES_DIR / f'activities_for_{top_file.stem}.md'
assert top_file.exists(), f'Top suggestion file not found: {top_file}'
assert agg_md.exists(), f'Aggregated activities file not found (run the activities cell first): {agg_md}'

# Read inputs
goals_text = top_file.read_text(encoding='utf-8')
activities_md = agg_md.read_text(encoding='utf-8')
curriculum_txt = CURRICULUM_PATH.read_text(encoding='utf-8')

# Styles
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name='TitleCenter', parent=styles['Title'], alignment=TA_CENTER, spaceAfter=12))
styles.add(ParagraphStyle(name='H2', parent=styles['Heading2'], spaceBefore=12, spaceAfter=6, textColor=colors.HexColor('#0b5394')))
styles.add(ParagraphStyle(name='H3', parent=styles['Heading3'], spaceBefore=10, spaceAfter=4, textColor=colors.HexColor('#38761d')))
styles.add(ParagraphStyle(name='Body', parent=styles['BodyText'], leading=14))
styles.add(ParagraphStyle(name='Small', parent=styles['BodyText'], fontSize=8, textColor=colors.grey))


def para(text, style='Body'):
    # Escape user/LLM text so stray < > & don't break the XML
    s = escape(text)
    # Minimal markdown → XML: **bold**, *italics*
    s = re.sub(r"\*\*(.+?)\*\*", r"<b>\1</b>", s)
    s = re.sub(r"(?<!\*)\*(?!\*)(.+?)(?<!\*)\*(?!\*)", r"<i>\1</i>", s)
    return Paragraph(s.replace("\n", "<br/>"), styles[style])


def para(text, style='Body'):
    # Light markdown → XML: bold **text**; italics *text*
    text = re.sub(r"\*\*(.+?)\*\*", r"<b>\1</b>", text)
    text = re.sub(r"(?<!\*)\*(?!\*)(.+?)(?<!\*)\*(?!\*)", r"<i>\1</i>", text)
    return Paragraph(text.replace('\n', '<br/>'), styles[style])

def parse_numbered_lines(text):
    items = []
    for line in text.splitlines():
        s = line.strip()
        if not s:
            continue
        m = re.match(r'^\d+\s*[\).:-]?\s+(.*)$', s)
        if m:
            items.append(m.group(1).strip())
    return items

def markdown_sections(md):
    sections = []
    current_h = None
    buf = []
    for line in md.splitlines():
        if line.startswith('## '):
            if current_h is not None:
                sections.append((current_h, '\n'.join(buf).strip()))
            current_h = line[3:].strip()
            buf = []
        else:
            buf.append(line)
    if current_h is not None:
        sections.append((current_h, '\n'.join(buf).strip()))
    return sections

def split_numbered_blocks(md):
    """Split a markdown numbered list into blocks (full text per item)."""
    lines = md.splitlines()
    blocks = []
    buf = []
    def is_start(l):
        return re.match(r'^\s*\d+\s*[\).:-]?\s+\S', l) is not None
    for i, line in enumerate(lines):
        if is_start(line):
            if buf:
                blocks.append('\n'.join(buf).strip())
                buf = []
            first = re.sub(r'^\s*\d+\s*[\).:-]?\s+', '', line, count=1)
            buf.append(first)
        else:
            buf.append(line)
    if buf:
        blocks.append('\n'.join(buf).strip())
    return [b for b in blocks if b.strip()]

def split_title_and_details(block):
    lines = [l for l in block.splitlines() if l.strip()]
    if not lines:
        return ("Aktivitet", "")
    title = lines[0].strip()
    details = '\n'.join(lines[1:]).strip()
    return (title, details)

# Build PDF
doc = SimpleDocTemplate(str(PDF_PATH), pagesize=A4,
                        leftMargin=2*cm, rightMargin=2*cm, topMargin=2*cm, bottomMargin=2*cm)
story = []

# Cover
story.append(para('Læringsmål og aktiviteter', 'TitleCenter'))
story.append(para(datetime.now().strftime('%Y-%m-%d %H:%M'), 'Small'))
story.append(Spacer(1, 12))
story.append(para(f"Generasjonsmodell: <b>{GENERATION_MODEL}</b> — Vurderingsmodell: <b>{JUDGE_MODEL}</b>", 'Small'))
story.append(Spacer(1, 18))
story.append(para('<b>Kompetansemål (kilde)</b>', 'H2'))
story.append(para(curriculum_txt, 'Body'))
story.append(PageBreak())

# Top-ranked goals
story.append(para(f'Topprangert læringsmålsliste (suggestion_{top_idx:02d}.txt) — score {top_score:.3f}', 'H2'))
goals_items = parse_numbered_lines(goals_text)
if goals_items:
    lst = ListFlowable([ListItem(para(it, 'Body')) for it in goals_items], bulletType='1', start='1', leftIndent=18)
    story.append(lst)
else:
    story.append(para(goals_text, 'Body'))

story.append(PageBreak())
story.append(para('Forslåtte læringsaktiviteter per læringsmål', 'H2'))

sections = markdown_sections(activities_md)
for heading, body in sections:
    story.append(para(heading, 'H3'))
    blocks = split_numbered_blocks(body)  # full block per numbered item
    if blocks:
        items = []
        for block in blocks:
            title, details = split_title_and_details(block)
            full_text = f"**{title}**"
            if details:
                full_text += "\n" + details  # keep all lines
            # IMPORTANT: no KeepTogether here
            items.append(ListItem(para(full_text, 'Body')))
        lst = ListFlowable(items, bulletType='1', start='1', leftIndent=18)
        story.append(lst)
    else:
        story.append(para(body, 'Body'))
    story.append(Spacer(1, 12))


doc.build(story)
print(f'PDF saved to: {PDF_PATH}')


PDF saved to: suggestions/report_learning_goals_activities.pdf


## Outputs
- `suggestions/suggestion_XX.txt` (each list)
- `suggestions/all_suggestions.txt`
- `suggestions/ranking.csv` and `ranking.json` (include **begrunnelse**)
- `suggestions/activities/*.md` (activities per goal and aggregated)
- `suggestions/report_learning_goals_activities.pdf` (full activity text)
